In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
client.delete_dataset(dataset_name=dataset_name)

In [8]:
from langsmith import Client
from langsmith.schemas import Run, Example
from langsmith.evaluation import evaluate
import openai
from langsmith.wrappers import wrap_openai

client = Client()

# Define dataset: these are your test cases
dataset_name = "Rap Battle Dataset"
dataset = client.create_dataset(dataset_name, description="Rap battle prompts.")
client.create_examples(
    inputs=[
        {"question": "石丸伸二と小池百合子のラップバトル"},
        {"question": "蓮舫と小池百合子のラップバトル"},
    ],
    outputs=[
        {"must_mention": ["安芸高田", "東京"]},
        {"must_mention": ["東京", "子育て"]},
    ],
    dataset_id=dataset.id,
)

# Define AI system
openai_client = wrap_openai(openai.Client())

def predict(inputs: dict) -> dict:
    messages = [{"role": "user", "content": inputs["question"]}]
    response = openai_client.chat.completions.create(messages=messages, model="gpt-4o")
    return {"output": response}

# Define evaluators
def must_mention(run: Run, example: Example) -> dict:
    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("must_mention") or []
    score = all(phrase in prediction for phrase in required)
    return {"key":"must_mention", "score": score}

experiment_results = evaluate(
    predict, # Your AI system
    data=dataset_name, # The data to predict and grade over
    evaluators=[must_mention], # The evaluators to score the results
    experiment_prefix="rap-generator", # A prefix for your experiment names to easily identify them
    metadata={
      "version": "1.0.0",
    },
)


View the evaluation results for experiment: 'rap-generator-7efa4a5d' at:
https://smith.langchain.com/o/bd14a154-65e7-52b4-bdce-b9a16d5e3513/datasets/d1a477ac-d044-410d-88d4-017a78c2a334/compare?selectedSessions=bbba7dc9-327a-43fc-9526-44858c62f504




0it [00:00, ?it/s]